# 🚀 George's Dream Factory - FLUX KREA Pro Edition\n\n**Optimized for Google Colab Pro** with T5-v1.1-XXL text encoder and advanced features\n\n## Features:\n- **T5-v1.1-XXL**: Google's best text encoder (11B parameters)\n- **CLIP ViT-L/14**: Additional text encoding for better understanding\n- **xFormers**: 50% memory reduction with faster attention\n- **Batch Generation**: Process multiple images at once\n- **Progress Tracking**: Real-time generation updates\n- **VAE Slicing**: Generate HD images up to 2048px\n\n## Setup Instructions:\n1. **Runtime**: Go to Runtime → Change runtime type → GPU → A100/V100\n2. **Run cells**: Execute all cells in order\n3. **Get URL**: Copy the ngrok URL from the output\n4. **Connect**: Paste URL in your frontend settings

In [ ]:
# Check GPU and system resources
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv
print("\n📊 System Information:")
!free -h
!df -h /content

In [ ]:
# Install optimized dependencies
print("📦 Installing dependencies optimized for Colab Pro...")
!pip install -q --upgrade pip

# PyTorch with CUDA 11.8
!pip install -q torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Core ML libraries
!pip install -q diffusers==0.26.3 transformers==4.38.1 accelerate==0.27.2 sentencepiece==0.1.99

# xFormers for memory optimization
!pip install -q xformers==0.0.23 --index-url https://download.pytorch.org/whl/cu118

# API and utilities
!pip install -q fastapi uvicorn pyngrok python-multipart pillow
!pip install -q safetensors einops omegaconf scipy ftfy beautifulsoup4

print("✅ All dependencies installed!")

In [ ]:
# Import libraries and set up environment
import os
import gc
import base64
import io
import json
import threading
import time
import queue
from datetime import datetime
from typing import Optional, List, Dict, Any
import uuid

import torch
import torch.nn as nn
from torch.cuda.amp import autocast
from PIL import Image
from fastapi import FastAPI, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
import uvicorn
from pyngrok import ngrok
import nest_asyncio
import numpy as np
from diffusers import FluxPipeline, DiffusionPipeline
from transformers import T5EncoderModel, T5TokenizerFast, CLIPTextModel, CLIPTokenizer
import warnings
warnings.filterwarnings('ignore')

# Allow nested event loops
nest_asyncio.apply()

print("🌟 FLUX KREA PRO - Enhanced with T5-XXL + CLIP")
print("=" * 60)

In [ ]:
# Configuration
class Config:
    # Model settings
    MODEL_ID = "black-forest-labs/FLUX.1-schnell"
    T5_MODEL = "google/t5-v1_1-xxl"  # Best T5 model
    CLIP_MODEL = "openai/clip-vit-large-patch14"
    
    # Server settings
    PORT = 7860
    ENABLE_XFORMERS = True
    ENABLE_VAE_SLICING = True
    ENABLE_CPU_OFFLOAD = False
    
    # Generation defaults (KREA optimized)
    DEFAULT_STEPS = 30
    DEFAULT_GUIDANCE = 4.0
    DEFAULT_WIDTH = 1024
    DEFAULT_HEIGHT = 1024
    MAX_BATCH_SIZE = 4
    
    # Performance
    USE_FLOAT16 = True
    TORCH_COMPILE = False  # Set True for 30% speedup after warmup
    
    # Your ngrok token (get free at https://ngrok.com)
    NGROK_TOKEN = None  # Replace with your token

config = Config()

# Set your ngrok token here or in the next cell
# config.NGROK_TOKEN = "your_token_here"

In [ ]:
# Enhanced Model Manager with T5-XXL
class FluxKreaModelManager:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.pipe = None
        self.t5_encoder = None
        self.t5_tokenizer = None
        self.clip_model = None
        self.clip_tokenizer = None
        self.model_loaded = False
        self.loading_progress = 0
        self.loading_status = "Not started"
        
    def load_t5_xxl(self):
        """Load the T5-XXL model for superior text encoding"""
        print("📝 Loading T5-v1.1-XXL text encoder (this is 11B parameters!)...")
        self.loading_status = "Loading T5-XXL encoder"
        self.loading_progress = 10
        
        try:
            # Load tokenizer
            self.t5_tokenizer = T5TokenizerFast.from_pretrained(
                config.T5_MODEL,
                model_max_length=512
            )
            
            # Load model with optimization
            self.t5_encoder = T5EncoderModel.from_pretrained(
                config.T5_MODEL,
                torch_dtype=torch.float16 if config.USE_FLOAT16 else torch.float32,
                device_map="auto",  # Automatically handle model sharding
                load_in_8bit=False,  # Set True if running out of memory
                low_cpu_mem_usage=True
            )
            
            self.loading_progress = 30
            print("✅ T5-XXL loaded successfully!")
            return True
            
        except Exception as e:
            print(f"❌ Error loading T5-XXL: {str(e)}")
            return False
    
    def load_clip(self):
        """Load CLIP model for additional text encoding"""
        print("🎨 Loading CLIP text encoder...")
        self.loading_status = "Loading CLIP encoder"
        self.loading_progress = 40
        
        try:
            self.clip_tokenizer = CLIPTokenizer.from_pretrained(config.CLIP_MODEL)
            self.clip_model = CLIPTextModel.from_pretrained(
                config.CLIP_MODEL,
                torch_dtype=torch.float16 if config.USE_FLOAT16 else torch.float32
            ).to(self.device)
            
            self.loading_progress = 50
            print("✅ CLIP loaded successfully!")
            return True
            
        except Exception as e:
            print(f"❌ Error loading CLIP: {str(e)}")
            return False
    
    def load_flux_pipeline(self):
        """Load FLUX pipeline with custom text encoders"""
        print("🔮 Loading FLUX pipeline with enhanced text encoders...")
        self.loading_status = "Loading FLUX model"
        self.loading_progress = 60
        
        try:
            # Load FLUX with our custom encoders
            self.pipe = FluxPipeline.from_pretrained(
                config.MODEL_ID,
                text_encoder=self.t5_encoder,
                text_encoder_2=self.clip_model,
                tokenizer=self.t5_tokenizer,
                tokenizer_2=self.clip_tokenizer,
                torch_dtype=torch.float16 if config.USE_FLOAT16 else torch.float32,
                use_safetensors=True,
                variant="fp16" if config.USE_FLOAT16 else None
            )
            
            self.pipe = self.pipe.to(self.device)
            self.loading_progress = 80
            
            # Apply optimizations
            if config.ENABLE_XFORMERS and self.device == "cuda":
                self.pipe.enable_xformers_memory_efficient_attention()
                print("⚡ xFormers enabled - 50% memory reduction!")
            
            if config.ENABLE_VAE_SLICING:
                self.pipe.enable_vae_slicing()
                print("🔪 VAE slicing enabled for HD images")
            
            # Compile with Torch 2.0 for speed
            if config.TORCH_COMPILE and hasattr(torch, 'compile'):
                print("🔥 Compiling model with Torch 2.0...")
                self.pipe.unet = torch.compile(self.pipe.unet, mode="reduce-overhead")
            
            self.loading_progress = 100
            self.loading_status = "Ready"
            self.model_loaded = True
            
            print("✅ FLUX pipeline ready with T5-XXL + CLIP!")
            return True
            
        except Exception as e:
            print(f"❌ Error loading FLUX: {str(e)}")
            self.loading_status = f"Error: {str(e)}"
            return False
    
    def load_all(self):
        """Load all models in sequence"""
        success = (
            self.load_t5_xxl() and
            self.load_clip() and
            self.load_flux_pipeline()
        )
        
        if success:
            # Optimize memory
            gc.collect()
            torch.cuda.empty_cache()
            
            print("\n🎉 All models loaded successfully!")
            print(f"💾 GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.1f}GB used")
        
        return success

# Initialize model manager
model_manager = FluxKreaModelManager()

In [ ]:
# Request models and queue system
generation_queue = queue.Queue()
results_cache = {}

class GenerationRequest:
    def __init__(self, request_id, prompt, params):
        self.id = request_id
        self.prompt = prompt
        self.params = params
        self.status = "queued"
        self.progress = 0
        self.result = None
        self.error = None
        self.created_at = time.time()

# Pydantic models for API
class GenerateRequest(BaseModel):
    prompt: str = Field(..., description="Text prompt")
    steps: int = Field(30, ge=20, le=50)
    cfg_guidance: float = Field(4.0, ge=1.0, le=10.0)
    seed: int = Field(-1)
    width: int = Field(1024, ge=512, le=2048)
    height: int = Field(1024, ge=512, le=2048)
    negative_prompt: str = Field("", description="What to avoid")

class BatchGenerateRequest(BaseModel):
    prompts: List[str] = Field(..., max_items=4)
    steps: int = Field(30)
    cfg_guidance: float = Field(4.0)
    width: int = Field(1024)
    height: int = Field(1024)
    negative_prompt: str = Field("")

In [ ]:
# Create FastAPI app with all endpoints
app = FastAPI(title="FLUX KREA Pro API", version="2.0")

# Configure CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def root():
    return {
        'name': 'FLUX KREA Pro API',
        'version': '2.0',
        'model': 'FLUX.1 with T5-XXL',
        'features': [
            'T5-v1.1-XXL text encoder (11B params)',
            'CLIP ViT-L/14 encoder',
            'Batch generation up to 4 images',
            'Real-time progress tracking',
            'xFormers memory optimization',
            'VAE slicing for HD images'
        ]
    }

@app.get("/health")
async def health_check():
    return {
        'status': 'healthy' if model_manager.model_loaded else 'loading',
        'device': model_manager.device,
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else None,
        'memory': {
            'allocated': f"{torch.cuda.memory_allocated() / 1024**3:.1f}GB",
            'reserved': f"{torch.cuda.memory_reserved() / 1024**3:.1f}GB"
        } if torch.cuda.is_available() else None
    }

@app.get("/status")
async def status_check():
    return {
        'status': model_manager.loading_status,
        'progress': model_manager.loading_progress,
        'model_loaded': model_manager.model_loaded,
        'models_loaded': model_manager.model_loaded,
        'models_loading': not model_manager.model_loaded and model_manager.loading_progress > 0,
        'queue_size': generation_queue.qsize(),
        'device': model_manager.device
    }

@app.post("/generate")
async def generate_image(request: GenerateRequest):
    if not model_manager.model_loaded:
        raise HTTPException(503, "Model is still loading")
    
    request_id = str(uuid.uuid4())
    gen_request = GenerationRequest(
        request_id=request_id,
        prompt=request.prompt,
        params=request.dict()
    )
    
    generation_queue.put(gen_request)
    results_cache[request_id] = gen_request
    
    # Wait for result (with timeout)
    timeout = 300  # 5 minutes
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        if gen_request.status == "completed":
            return {
                'success': True,
                'image': gen_request.result['images'][0],
                'seed': gen_request.result['seed'],
                'duration': gen_request.result['duration']
            }
        elif gen_request.status == "failed":
            raise HTTPException(500, gen_request.error)
        await asyncio.sleep(0.1)
    
    raise HTTPException(504, "Generation timeout")

@app.post("/generate/batch")
async def generate_batch(request: BatchGenerateRequest):
    if not model_manager.model_loaded:
        raise HTTPException(503, "Model is still loading")
    
    request_ids = []
    for prompt in request.prompts:
        request_id = str(uuid.uuid4())
        params = request.dict()
        params['prompt'] = prompt
        
        gen_request = GenerationRequest(
            request_id=request_id,
            prompt=prompt,
            params=params
        )
        
        generation_queue.put(gen_request)
        results_cache[request_id] = gen_request
        request_ids.append(request_id)
    
    return {
        'success': True,
        'request_ids': request_ids,
        'message': f'Queued {len(request.prompts)} images'
    }

@app.get("/progress/{request_id}")
async def get_progress(request_id: str):
    if request_id not in results_cache:
        raise HTTPException(404, "Invalid request ID")
    
    gen_request = results_cache[request_id]
    
    response = {
        'request_id': request_id,
        'status': gen_request.status,
        'progress': gen_request.progress
    }
    
    if gen_request.status == "completed":
        response['result'] = gen_request.result
    elif gen_request.status == "failed":
        response['error'] = gen_request.error
    
    return response

@app.get("/models")
async def get_models():
    return {
        'models': [{
            'id': 'flux-krea-pro',
            'name': 'FLUX KREA Pro with T5-XXL',
            'status': 'loaded' if model_manager.model_loaded else 'loading',
            'text_encoders': {
                't5': 'google/t5-v1_1-xxl (11B params)',
                'clip': 'openai/clip-vit-large-patch14'
            }
        }]
    }

In [ ]:
# Generation worker function
def generation_worker():
    """Process generation requests from queue"""
    while True:
        try:
            gen_request = generation_queue.get(timeout=1)
            
            if gen_request is None:
                break
            
            print(f"\n🎨 Generating: {gen_request.prompt[:60]}...")
            gen_request.status = "processing"
            gen_request.progress = 10
            
            # Set up parameters
            params = gen_request.params
            
            # Handle seed
            if params['seed'] == -1:
                params['seed'] = torch.randint(0, 1000000, (1,)).item()
            
            generator = torch.Generator(device=model_manager.device).manual_seed(params['seed'])
            
            # Generate
            start_time = time.time()
            
            try:
                with torch.cuda.amp.autocast(enabled=True):
                    gen_request.progress = 50
                    
                    # Generate with T5-XXL enhanced prompts
                    result = model_manager.pipe(
                        prompt=params['prompt'],
                        negative_prompt=params.get('negative_prompt', ''),
                        num_inference_steps=params['steps'],
                        guidance_scale=params['cfg_guidance'],
                        generator=generator,
                        width=params['width'],
                        height=params['height']
                    )
                    
                    gen_request.progress = 90
                    
                    # Convert to base64
                    images = []
                    for img in result.images:
                        buffered = io.BytesIO()
                        img.save(buffered, format="PNG", optimize=True)
                        img_base64 = base64.b64encode(buffered.getvalue()).decode()
                        images.append(f"data:image/png;base64,{img_base64}")
                    
                    duration = time.time() - start_time
                    
                    gen_request.result = {
                        'images': images,
                        'seed': params['seed'],
                        'duration': duration
                    }
                    gen_request.status = "completed"
                    gen_request.progress = 100
                    
                    print(f"✅ Generated in {duration:.1f}s | Seed: {params['seed']}")
                    
            except Exception as e:
                gen_request.error = str(e)
                gen_request.status = "failed"
                print(f"❌ Error: {str(e)}")
            
            # Cleanup
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
        except queue.Empty:
            continue
        except Exception as e:
            print(f"Worker error: {e}")

In [ ]:
# Load models in background
print("🔄 Starting model loading...")
print("This will load:")
print("  • T5-v1.1-XXL (11B parameters)")
print("  • CLIP ViT-L/14")
print("  • FLUX pipeline")
print("\nThis may take 3-5 minutes...\n")

def load_models_background():
    success = model_manager.load_all()
    if success:
        print("\n✨ FLUX KREA Pro is ready!")
        print("=" * 60)
        print("🚀 Enhanced with:")
        print("  • T5-v1.1-XXL for superior text understanding")
        print("  • xFormers for 50% memory savings")
        print("  • Batch processing support")
        print("  • Real-time progress tracking")
        print("=" * 60)

# Start loading
model_thread = threading.Thread(target=load_models_background, daemon=True)
model_thread.start()

In [ ]:
# Set up ngrok (optional but recommended)
# Get your free auth token at: https://dashboard.ngrok.com/signup

# Uncomment and add your token:
# ngrok.set_auth_token("your_ngrok_auth_token_here")

# Or set it in environment
# os.environ['NGROK_AUTH_TOKEN'] = "your_token"

In [ ]:
# Start everything!
print("🌐 Setting up public access...")

# Create ngrok tunnel
public_url = ngrok.connect(config.PORT)
print(f"\n🚀 FLUX KREA Pro API is starting!")
print(f"\n🔗 Public URL: {public_url}")
print(f"📊 Status: {public_url}/status")
print(f"🏥 Health: {public_url}/health")
print(f"📚 Models: {public_url}/models")
print("\n" + "="*60)
print("🎯 CONNECTION INSTRUCTIONS:")
print("1. Wait for models to load (check /status)")
print("2. Copy the public URL above")
print("3. Open your Dream Factory frontend")
print("4. Paste the URL in settings")
print("5. Start creating amazing images!")
print("="*60)

# Start generation worker
worker_thread = threading.Thread(target=generation_worker, daemon=True)
worker_thread.start()

# Run the server
print("\n🚀 Starting API server...")
print("⚠️  Keep this cell running!\n")

config_uvicorn = uvicorn.Config(app, host="0.0.0.0", port=config.PORT, log_level="info")
server = uvicorn.Server(config_uvicorn)
await server.serve()

## 🧪 Test the API

Run these cells to test your API:

In [ ]:
# Test single generation
import requests
from IPython.display import Image as IPImage, display

# Use the public URL from above
test_url = str(public_url)  # or "http://localhost:7860" for local testing

# Check status first
status = requests.get(f"{test_url}/status").json()
print("Status:", status)

if status['model_loaded']:
    # Test generation
    response = requests.post(
        f"{test_url}/generate",
        json={
            "prompt": "A majestic snow leopard resting on a cliff at sunset, golden hour lighting, photorealistic, 8k quality",
            "steps": 30,
            "cfg_guidance": 4.0,
            "width": 1280,
            "height": 1024,
            "negative_prompt": "blurry, low quality, distorted"
        }
    )
    
    result = response.json()
    if result.get('success'):
        print(f"✅ Generated in {result['duration']:.1f}s")
        print(f"Seed: {result['seed']}")
        
        # Display the image
        img_data = base64.b64decode(result['image'].split(',')[1])
        display(IPImage(img_data))
    else:
        print("Error:", result)
else:
    print("⏳ Model is still loading...")

In [ ]:
# Test batch generation
if model_manager.model_loaded:
    response = requests.post(
        f"{test_url}/generate/batch",
        json={
            "prompts": [
                "A serene Japanese garden with cherry blossoms",
                "A futuristic cyberpunk cityscape at night",
                "An enchanted forest with bioluminescent plants"
            ],
            "steps": 28,
            "cfg_guidance": 3.5,
            "width": 1024,
            "height": 1024
        }
    )
    
    batch_result = response.json()
    print("Batch result:", batch_result)
    
    # Check progress
    if batch_result.get('success'):
        for req_id in batch_result['request_ids']:
            progress = requests.get(f"{test_url}/progress/{req_id}").json()
            print(f"Request {req_id}: {progress['status']} ({progress['progress']}%)")

## 📚 Tips & Best Practices

### Optimal Settings for FLUX KREA:
- **Steps**: 28-32 (sweet spot for quality/speed)
- **CFG Guidance**: 3.5-5.0 (lower than typical)
- **Resolution**: 1024-1280px (best quality)

### T5-XXL Prompt Tips:
- Be descriptive and specific
- Include style, lighting, and mood
- T5 understands complex relationships
- Use natural language, not just tags

### Performance Optimization:
- Enable xFormers (already enabled)
- Use batch generation for multiple images
- Keep resolution at 1024-1280 for best results
- Reuse seeds for consistent styles

### Troubleshooting:
- If OOM: Reduce batch size or resolution
- If slow: Check GPU type (A100 is best)
- If quality issues: Increase steps to 32-40
- If connection drops: Restart ngrok cell